<a href="https://colab.research.google.com/github/enessibest/Bagging-and-Boosting/blob/main/Bagging_and_boosting_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data= pd.read_csv('/content/talking_data.csv')
data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [5]:
data.shape

(100000, 8)

In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
ip,100000.0,91255.87967,69835.553661,9.0,40552.0,79827.0,118252.0,364757.0
app,100000.0,12.04788,14.941500,1.0,3.0,12.0,15.0,551.0
device,100000.0,21.77125,259.667767,0.0,1.0,1.0,1.0,3867.0
os,100000.0,22.81828,55.943136,0.0,13.0,18.0,19.0,866.0
channel,100000.0,268.83246,129.724248,3.0,145.0,258.0,379.0,498.0
is_attributed,100000.0,0.00227,0.047591,0.0,0.0,0.0,0.0,1.0


In [7]:
missing_value= data.isna().sum()
missing_value

ip                     0
app                    0
device                 0
os                     0
channel                0
click_time             0
attributed_time    99773
is_attributed          0
dtype: int64

In [8]:
data= data.drop(['attributed_time'], axis=1)

In [9]:
data.head()

,ip,app,device,os,channel,click_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,0
1,105560,25,1,17,259,2017-11-07 13:40:27,0
2,101424,12,1,19,212,2017-11-07 18:05:24,0
3,94584,13,1,13,477,2017-11-07 04:58:08,0
4,68413,12,1,1,178,2017-11-09 09:00:09,0


In [10]:
len(data.ip.unique())

34857

In [11]:
len(data.app.unique())

161

In [12]:
len(data.device.unique())

100

In [13]:
len(data.os.unique())

130

In [14]:
len(data.channel.unique())

161

In [15]:
##clcik time has the maximum unique value
len(data.click_time.unique())

80350

In [16]:
len(data.is_attributed.unique())

2

In [17]:
data.app.value_counts()

3      18279
12     13198
2      11737
9       8992
15      8595
       ...  
407        1
145        1
271        1
398        1
92         1
Name: app, Length: 161, dtype: int64

In [18]:
app_data= data['app'].value_counts()>2000
app_data

3       True
12      True
2       True
9       True
15      True
       ...  
407    False
145    False
271    False
398    False
92     False
Name: app, Length: 161, dtype: bool

In [19]:
new_app=app_data[app_data==True]
new_app

3     True
12    True
2     True
9     True
15    True
18    True
14    True
1     True
13    True
8     True
Name: app, dtype: bool

In [20]:
##4
def time_features(data):
  data['datetime']=pd.to_datetime(data['click_time'])
  data['day_of_week']=data['datetime'].dt.dayofweek
  data['day_of_year']=data['datetime'].dt.dayofyear
  data['month']=data['datetime'].dt.month
  data['hour']=data['datetime'].dt.hour
  return data

In [21]:
data=time_features(data)
data.head()

,ip,app,device,os,channel,click_time,is_attributed,datetime,day_of_week,day_of_year,month,hour
0,87540,12,1,13,497,2017-11-07 09:30:38,0,2017-11-07 09:30:38,1,311,11,9
1,105560,25,1,17,259,2017-11-07 13:40:27,0,2017-11-07 13:40:27,1,311,11,13
2,101424,12,1,19,212,2017-11-07 18:05:24,0,2017-11-07 18:05:24,1,311,11,18
3,94584,13,1,13,477,2017-11-07 04:58:08,0,2017-11-07 04:58:08,1,311,11,4
4,68413,12,1,1,178,2017-11-09 09:00:09,0,2017-11-09 09:00:09,3,313,11,9


In [22]:
data= data.drop(['click_time'], axis=1)
data= data.drop(['datetime'], axis =1)

In [23]:
data.head()

,ip,app,device,os,channel,is_attributed,day_of_week,day_of_year,month,hour
0,87540,12,1,13,497,0,1,311,11,9
1,105560,25,1,17,259,0,1,311,11,13
2,101424,12,1,19,212,0,1,311,11,18
3,94584,13,1,13,477,0,1,311,11,4
4,68413,12,1,1,178,0,3,313,11,9


In [24]:
from sklearn.model_selection import train_test_split
X=data.drop(['is_attributed'],axis = 1)
Y= data['is_attributed']


In [25]:
train_X,test_X, train_Y, test_Y=train_test_split(X, Y, test_size=0.2)

In [26]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

In [32]:
model=XGBClassifier()
model.fit(train_X, train_Y)
test_model=model.predict(test_X)
model_score = model.score(test_X,test_Y)

In [33]:
print(model_score)

0.99755


In [35]:
y_pred = model.predict_proba(test_X)
y_pred

array([[9.9980980e-01, 1.9020138e-04],
       [9.9950677e-01, 4.9320288e-04],
       [9.9940377e-01, 5.9619552e-04],
       ...,
       [9.9984145e-01, 1.5855214e-04],
       [9.9969900e-01, 3.0099979e-04],
       [9.9981368e-01, 1.8634477e-04]], dtype=float32)

In [38]:
##AUC ,area under the curve
from sklearn import metrics
ROC=metrics.roc_auc_score(test_Y,y_pred[:,1])
print("AUC: %.2f%%" % (ROC * 100.0))


AUC: 92.31%


## USING BAGGING CLASSIFIER 

In [39]:
from sklearn.ensemble import BaggingClassifier
Bgcl=BaggingClassifier(n_estimators=10, max_samples=0.7,bootstrap=True)
Bgcl.fit(train_X, train_Y)
test_pred = Bgcl.predict(test_X)
Bgcl_score=Bgcl.score(test_X ,test_Y)
print("The Bagging classifier model accuracy is {}".format(Bgcl_score))



The Bagging classifier model accuracy is 0.99755


In [42]:
y_pred1 = Bgcl.predict_proba(test_X)
y_pred1

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [43]:
##AUC ,area under the curve
from sklearn import metrics
ROC=metrics.roc_auc_score(test_Y,y_pred1[:,1])
print("AUC: %.2f%%" % (ROC * 100.0))

AUC: 81.47%
